# インタラクティブA/Bテスト分析ツール

このノートブックでは、サンプル数と成功数を入力するだけで、ベイジアンA/Bテストと頻度主義A/Bテストの両方の結果を可視化できます。

## 使い方

1. 下記のセルを順番に実行してください
2. 「データ入力」セクションで、グループAとグループBのサンプル数と成功数を入力してください
3. 分析を実行すると、分布表と可視化グラフが表示されます

## セットアップ

In [ ]:
# 必要なライブラリのインポート
import sys
sys.path.append('..')

from src.test_data import TestData, TestMethod
from src.bayesian import BayesianABTest
from src.frequentist import FrequentistABTest
from src.visualization import (
    plot_bayesian_distributions,
    plot_frequentist_results,
    plot_comparison,
    create_distribution_table
)

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Jupyter Notebook内でグラフを表示
%matplotlib inline

print("✅ セットアップ完了！")

## データ入力

下記のセルでグループAとグループBのデータを入力してください。

- `n_a`: グループAのサンプル数（例: 訪問者数）
- `conv_a`: グループAの成功数（例: コンバージョン数）
- `n_b`: グループBのサンプル数
- `conv_b`: グループBの成功数

In [ ]:
# ========================================
# ここにデータを入力してください
# ========================================

# グループA
n_a = 1000      # サンプル数
conv_a = 120    # 成功数

# グループB
n_b = 1000      # サンプル数
conv_b = 150    # 成功数

# ========================================

# データオブジェクトの作成
try:
    data = TestData(n_a=n_a, conv_a=conv_a, n_b=n_b, conv_b=conv_b)
    print("✅ データが正常に読み込まれました！")
    print(f"\nグループA: CVR = {data.cvr_a:.2%} ({conv_a}/{n_a})")
    print(f"グループB: CVR = {data.cvr_b:.2%} ({conv_b}/{n_b})")
    print(f"差分: {data.cvr_diff:+.2%}")
except ValueError as e:
    print(f"❌ エラー: {e}")
    print("データを確認して再実行してください。")

## A/Bテスト分析の実行

In [ ]:
# ベイジアンA/Bテストの実行
bayesian_test = BayesianABTest(
    data=data,
    alpha_prior=1.0,  # 無情報事前分布
    beta_prior=1.0,
    credible_level=0.95,
    n_samples=100000
)
bayesian_result = bayesian_test.run()

# 頻度論的A/Bテストの実行
frequentist_test = FrequentistABTest(
    data=data,
    confidence_level=0.95
)
frequentist_result = frequentist_test.run(TestMethod.Z_TEST)

print("✅ 分析完了！")

## 結果の表示

### 分布表

In [ ]:
# 分布表の表示
table = create_distribution_table(data, bayesian_result, frequentist_result)
print(table)

### ベイジアンA/Bテストの可視化

In [ ]:
# ベイジアン分布の可視化
fig_bayesian = plot_bayesian_distributions(bayesian_test, bayesian_result, figsize=(14, 10))
plt.show()

### 頻度論的A/Bテストの可視化

In [ ]:
# 頻度論的結果の可視化
fig_frequentist = plot_frequentist_results(data, frequentist_result, figsize=(14, 8))
plt.show()

### ベイジアンvs頻度論的の比較

In [ ]:
# 両アプローチの比較
fig_comparison = plot_comparison(data, bayesian_result, frequentist_result, figsize=(16, 6))
plt.show()

## サンプルデータで試してみる

以下は、いくつかのサンプルシナリオです。興味があれば試してみてください！

### シナリオ1: 明らかな差がある場合

In [ ]:
# グループA: 10% CVR, グループB: 15% CVR
data_scenario1 = TestData(n_a=2000, conv_a=200, n_b=2000, conv_b=300)

bayesian_test1 = BayesianABTest(data_scenario1, credible_level=0.95)
bayesian_result1 = bayesian_test1.run()

frequentist_test1 = FrequentistABTest(data_scenario1, confidence_level=0.95)
frequentist_result1 = frequentist_test1.run(TestMethod.Z_TEST)

print("\n=== シナリオ1: 明らかな差がある場合 ===")
print(create_distribution_table(data_scenario1, bayesian_result1, frequentist_result1))

fig = plot_comparison(data_scenario1, bayesian_result1, frequentist_result1)
plt.show()

### シナリオ2: 差がほとんどない場合

In [ ]:
# グループA: 10% CVR, グループB: 10.5% CVR
data_scenario2 = TestData(n_a=1000, conv_a=100, n_b=1000, conv_b=105)

bayesian_test2 = BayesianABTest(data_scenario2, credible_level=0.95)
bayesian_result2 = bayesian_test2.run()

frequentist_test2 = FrequentistABTest(data_scenario2, confidence_level=0.95)
frequentist_result2 = frequentist_test2.run(TestMethod.Z_TEST)

print("\n=== シナリオ2: 差がほとんどない場合 ===")
print(create_distribution_table(data_scenario2, bayesian_result2, frequentist_result2))

fig = plot_comparison(data_scenario2, bayesian_result2, frequentist_result2)
plt.show()

### シナリオ3: サンプルサイズが小さい場合

In [ ]:
# グループA: 10% CVR, グループB: 20% CVR (小サンプル)
data_scenario3 = TestData(n_a=100, conv_a=10, n_b=100, conv_b=20)

bayesian_test3 = BayesianABTest(data_scenario3, credible_level=0.95)
bayesian_result3 = bayesian_test3.run()

frequentist_test3 = FrequentistABTest(data_scenario3, confidence_level=0.95)
frequentist_result3 = frequentist_test3.run(TestMethod.Z_TEST)

print("\n=== シナリオ3: サンプルサイズが小さい場合 ===")
print(create_distribution_table(data_scenario3, bayesian_result3, frequentist_result3))

fig = plot_comparison(data_scenario3, bayesian_result3, frequentist_result3)
plt.show()

## まとめ

このノートブックを使用すると:

1. **ベイジアンアプローチ**: 確率的な推論により、BがAより優れている確率を直接計算できます
2. **頻度論的アプローチ**: 統計的仮説検定により、帰無仮説を棄却できるかを判定できます
3. **両者の比較**: 同じデータに対する2つのアプローチの結果を比較できます

### ベイジアンと頻度論的の違い

- **ベイジアン**: 「BがAより優れている確率は95%」という直感的な解釈が可能
- **頻度論的**: 「帰無仮説（差がない）を棄却できるか」を検定（p値 < 0.05で有意）

どちらを使用するかは、ビジネスコンテキストや意思決定の性質によって異なります。